In [ ]:
# Importing all necessaries
from collections import deque
import sys
import math
import numpy as np
import pandas as pd
import datetime

# We look to store the rwds and the best of them that we obtain per 100 eps in a dataframe
# This refers to a labelled tabular representation

def save_rwds_csv(avg_rwd_100, best_avg_rwd_100):
    eps = np.arange(1, len(avg_rwd_100) + 1)
    rwd_per_eps = pd.DataFrame({
        'avg_rwd_100': avg_rwd_100,
        'best_avg_rwd_100': best_avg_rwd_100,
        'eps': eps
    })

# Dataframe is then converted to csv for analysis    
    rwd_per_eps.to_csv('rwds_plot_data.csv')

# Interact function is defined
# This is to monitor the performance of the agent
# There are certain parameters regarding monitoring the performance
# env denoting instance of the environment
#agent dentoing instance of class Agent that was created
#num_eps denoting number of eps of agent-environment interacn
#window representing the number of eps to be considered for mamnipulating average rwds

def interact(env, agent, num_eps=50000, window=100):
    
    # we initialize average rwds
    avg_rwd_100 = []
    best_avg_rwd_100 = []
    avg_rwds = deque(maxlen=num_eps)
    # we initialize best average rwd
    best_avg_rwd = -math.inf
    # then we initialize monitor 
    # for most recent rwds
    sample_rwd = deque(maxlen=window)
    
    
    # For each episode we run:
    for i_episode in range(1, num_eps+1):
        state = env.reset()
        # initializing the sampled rwd
        sample_rwd = 0
        while True:
            # agent selects an acn(action)
            # agent then is set to perform the selected acn(action)
            acn = agent.select_acn(state)
            nxt_st, rwd, done, _ = env.step(acn)

            # updating the sampled rwd
            agent.step(state, acn, rwd, nxt_st, done)
            sample_rwd += rwd
            # updating the state to next time step
            state = nxt_st
            agent.update_epsilon()
            if done:
                sample_rwds.append(sample_rwd)
                break

# we want to get average rwd from last 100 eps and update the sampled rwd
        if (i_episode >= 100):
            avg_rwd = np.mean(sample_rwds)
            avg_rwds.append(avg_rwd)
           
            print('episode average rwd {}'.format(avg_rwd))
            avg_rwd_100.append(avg_rwd)
            best_avg_rwd_100.append(best_avg_rwd)
            if avg_rwd > best_avg_rwd:
                best_avg_rwd = avg_rwd
                
        # print out progress in related to average rwd obtained
        print("\rEpisode {}/{} || Best average rwd {} || eps {} ".format(i_episode, num_eps, best_avg_rwd, agent.epsilon), end="")
        sys.stdout.flush()
        
        # saving the best onbtained rwd to the csv we have already created
        # and end execution after average rwds and best of them are obtained and gathered
        if best_avg_rwd >= 9.7:
            print('\nEnvironment solved in {} eps.'.format(i_episode), end="")
            agent.q_table.save(best_avg_rwd)
            save_rwds_csv(avg_rwd_100, best_avg_rwd_100)
            break
        if i_episode == num_eps: 
            agent.q_table.save(best_avg_rwd)
            save_rwds_csv(avg_rwd_100, best_avg_rwd_100)
            print('\n')
    return avg_rwds, best_avg_rwd